In [ ]:
from common import *
from snotel_sites import SNOTEL
from dask_cluster import start_cluster
import dask

In [ ]:
client = start_cluster(4, 8)
client

In [ ]:
def write_day(date):
    comp = dict(zlib=True, complevel=5)

    home = PurePath(f'{Path.home()}/shared-cyrosphere/Rittger_albedo/')
    year = f"{date.year}-vrt"

    date_str = date.strftime('%Y%m%d')
    file = f"WesternUS_{date_str}_albedo_muz_32613.vrt"

    day = xr.open_rasterio(
        home.joinpath(year, file).as_posix()
    )
    day = day.expand_dims({'time': [date]})
    day.attrs['scale_factor'] = 0.0001
    del day.attrs['scales']
    del day.attrs['offsets']
    del day.attrs['is_tiled']
    day.name = 'albedo_muz'
    day = day.to_dataset()
    
    encoding = {var: comp for var in day.data_vars}

    day.to_netcdf(
        home.joinpath(f"{date.year}-nc", f"WesternUS_{date_str}.nc").as_posix(), 
        encoding=encoding,
    )
    del day

In [ ]:
dask_queue = []

for date in pd.date_range(start='2019-01-01', end='2019-09-30'):   
    dask_queue.append(
        dask.delayed(write_day)(date)
    )

In [ ]:
dask.compute(dask_queue);